In [1]:
## Importing Libraries
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Read data
data = input_data.read_data_sets("MNIST", one_hot=True)

Extracting MNIST\train-images-idx3-ubyte.gz
Extracting MNIST\train-labels-idx1-ubyte.gz
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz


In [8]:
# Placeholder for noisy image 
img_noisy = tf.placeholder(tf.float32, [None, 784])

# Placeholder for original image
img_original = tf.placeholder(tf.float32, [None, 784])

# Input Layer
input_layer = tf.reshape(img_noisy, [-1, 28, 28, 1])

print(input_layer.shape)

# Convolutional Layer #1
conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=(5, 5), padding="same", activation=tf.nn.relu)

print(conv1.shape)

# Pooling Layer #1
#pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
#print(pool1.shape)

# Convolutional Layer #2 and Pooling Layer #2
conv2 = tf.layers.conv2d(inputs=conv1, filters=64, kernel_size=(5, 5), padding="same", activation=tf.nn.relu)
#pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

print("conv2", conv2.shape)
#print(pool2.shape)

# Fully-connected layer 1
conv2_flat = tf.reshape(conv2, [-1, 28 * 28 * 64])

print(conv2_flat.shape)

dense1 = tf.layers.dense(inputs=conv2_flat, units=4*4*64, activation=tf.nn.relu)

print(dense1.shape)

dense2 = tf.layers.dense(inputs=dense1, units=28*28*64, activation=tf.nn.relu)

print(dense2.shape)

dense_reshaped = tf.reshape(dense2, [-1, 28, 28, 64])

print(dense_reshaped.shape)

conv_trans_1 = tf.layers.conv2d_transpose(dense_reshaped,
                                          filters=32,
                                          kernel_size=(5, 5),
                                          strides=(1, 1),
                                          padding="same",
                                          activation=tf.nn.relu)

print(conv_trans_1.shape)

conv_trans_2 = tf.layers.conv2d_transpose(dense_reshaped,
                                          filters=1,
                                          kernel_size=(5, 5),
                                          strides=(1, 1),
                                          padding="same",
                                          activation=tf.nn.relu)

print(conv_trans_2.shape)

# Calculate loss
loss = tf.reduce_mean(tf.subtract(conv_trans_2, img_original))

(?, 28, 28, 1)
(?, 28, 28, 32)
conv2 (?, 28, 28, 64)
(?, 50176)
(?, 1024)
(?, 50176)
(?, 28, 28, 64)
(?, 28, 28, 32)
(?, 28, 28, 1)


In [ ]:
optim = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Make batches to train
num_iter = 1000
batch_size = 64
for i in range(num_iter):
    batch_img, _ = data.train.next_batch(batch_size)
    
    # Add noise to the batch
    batch_img_noisy = np.copy(batch_img) + np.random.normal(loc=0.0, scale=0.15, size=batch_img.shape)
    
    _, l = sess.run([optim, loss], feed_dict={ img_noisy: batch_img_noisy, img_original: batch_img })
    print(l)